<a target="_blank" href="https://colab.research.google.com/github/Sebelino/DD2424-project/blob/main/e1_sebastian.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
%load_ext autoreload
%autoreload 2

import os
import socket

repo_name = "DD2424-project"
dep_filename = "download_dataset.py"
cwd = os.getcwd()
hostname = socket.gethostname()

print(f"Host: {hostname}")
print(f"You are here: {cwd}")

if os.path.exists(dep_filename):
    print(f"{dep_filename} is in your current directory. Assuming you are in the root of the repository.")
elif not os.path.exists(repo_name):
    print(f"Cloning repo and fetching {dep_filename}...")
    !git clone https://github.com/Sebelino/DD2424-project
    !cp DD2424-project/download_dataset.py .
else:
    !git -C DD2424-project pull --rebase
    !cp DD2424-project/download_dataset.py .

Host: deeplearning-vm
You are here: /home/silpasoninallacheruvu/DD2424-project
download_dataset.py is in your current directory. Assuming you are in the root of the repository.


# Train model

In [ ]:
# from dataclasses import asdict
# from util import json_dumps
# from evaluation import evaluate_test_accuracy_and_misclassified, \
#     evaluate_with_train_val_plot, run_with_different_seeds, run_comparison
# from run import run
# %reload_ext autoreload
# %autoreload 2

# from determinism import Determinism

# determinism = Determinism(seed=42).sow()  # Keep this before any torch import

# from training import TrainParams, NagParams, AdamParams

# baseline_params = TrainParams(
#     seed=determinism.seed,
#     batch_size=32,
#     architecture="resnet50",
#     n_epochs=10,
#     optimizer=NagParams(
#         learning_rate=0.01,
#         weight_decay=1e-4,
#         momentum=0.9,
#     ),
#     freeze_layers=True,
#     unfreezing_epochs=None,
#     validation_freq=1,
#     unfreeze_last_l_blocks = 1,
#     time_limit_seconds=None,
#     val_acc_target=None,
# )

# invalidate_cache_entry = True
# if invalidate_cache_entry:
#     run.call_and_shelve(baseline_params).clear()

# # First plot: Training and validation accuracy
# for l in range(1,6):
#     baseline_params.unfreeze_last_l_blocks = l
#     print("Training with params:")
#     print(baseline_params.pprint())
#     result = run(baseline_params)

#     print("Training result:")
#     print(result.pprint())
#     evaluate_with_train_val_plot(result)

#     # Second plot: Rerun training multiple times with different seed
#     run_with_different_seeds(baseline_params, trials=3)

#     # Third plot: Compare NAG with Adam
#     experimental_params = baseline_params.copy()
#     experimental_params.optimizer = AdamParams(learning_rate=0.001)
    
#     param_sets = {
#         "Baseline (NAG)": baseline_params,
#         "Experiment (Adam)": experimental_params,
#     }
    
#     run_comparison(param_sets, trials=3)


# Train semi-supervised model

In [ ]:
%reload_ext autoreload
%autoreload 2

from training import TrainParams, NagParams
from evaluation import evaluate_test_accuracy_and_misclassified, \
    evaluate_with_train_val_plot, run_with_different_seeds, run_comparison
from run import run
from caching import invalidate_cache_entry
from datasets import DatasetParams
from augmentation import AugmentationParams

from determinism import Determinism

determinism = Determinism(seed=42).sow()  # Keep this before any torch import
   
dataset_params = DatasetParams(
    splitting_seed=determinism.seed,
    shuffler_seed=determinism.seed,
    batch_size=32,
    #trainval_size=None,  # Load all training data
    class_fractions=(1.0,) * 37,
    #trainval_size=400,  # Load a subset
    validation_set_fraction=0.2,  # 20 % of trainval set
    labelled_data_fraction=0.5, # full labelled data first
)

baseline_params = TrainParams(
    seed=determinism.seed,
    architecture="resnet50",
    n_epochs=5,
    optimizer=NagParams(
        learning_rate=0.004713842776302303,
        weight_decay=4.287389519996026e-06,
        momentum=0.7222321829989696,
    ),
    freeze_layers=True,
    unfreezing_epochs=(2, 4),
    validation_freq=1,
    time_limit_seconds=None,
    val_acc_target=None,
    unfreeze_last_l_blocks = 2,
    pseudo_threshold = 0.95,
    unsup_weight = 1,
    augmentation = AugmentationParams(
        enabled=False,
        transform=None
    ),
    fixmatch=True,
    use_scheduler=True,
    scheduler_type="plateau"
)

run_args = (dataset_params, baseline_params)

# Set invalidate=True to invalidate a single cache entry
invalidate_cache_entry(run, run_args, invalidate=False)

# First plot: Training and validation accuracy
print("Training with params:")
print(baseline_params.pprint())
result = run(*run_args)

print("Training result:")
print(result.pprint())
evaluate_with_train_val_plot(result)

In [ ]:
from evaluation import run_dataset_comparison

def tweak(params: DatasetParams, labelled_data_fraction):
    params = params.copy()
    params.labelled_data_fraction = labelled_data_fraction
    return params
 
#plot: Compare unsupervised learning for different fractions of unlabelled data
labelled_data_fractions = [ 0.1, 0.3, 0.5, 0.7, 0.9, 1.0]
param_sets = {f"labelled_data_fraction={data_fraction}": tweak(dataset_params, labelled_data_fraction=data_fraction) for data_fraction in labelled_data_fractions}

run_dataset_comparison(param_sets, baseline_params)


In [ ]:
from evaluation import run_dataset_comparison

def tweak(params: DatasetParams, labelled_data_fraction):
    params = params.copy()
    params.labelled_data_fraction = labelled_data_fraction
    return params
 
#plot: Compare unsupervised learning for different fractions of unlabelled data along with threshold
labelled_data_fractions = [ 0.1, 0.3, 0.5, 0.7, 0.9, 1.0]
param_sets = {f"labelled_data_fraction={data_fraction}": tweak(dataset_params, labelled_data_fraction=data_fraction) for data_fraction in labelled_data_fractions}

baseline_params.pseudo_threshold = None
run_dataset_comparison(param_sets, baseline_params)


In [12]:
from evaluation import evaluate_test_accuracy
from run import try_loading_trainer
from augmentation import AugmentationParams
from evaluation import evaluate_final_test_accuracy, run_with_different_seeds
from torch.utils.data import DataLoader
from datasets import load_dataset, DatasetParams
from training import Trainer, TrainParams, NagParams
from copy import deepcopy
from caching import invalidate_cache_entry
from determinism import Determinism

determinism = Determinism(seed=42).sow()  # Keep this before any torch import

dataset_params = DatasetParams(
    splitting_seed=determinism.seed,
    shuffler_seed=determinism.seed,
    batch_size=32,
    #trainval_size=None,  # Load all training data
    class_fractions=(1.0,) * 37,
    #trainval_size=400,  # Load a subset
    validation_set_fraction=0.2,  # 20 % of trainval set
    labelled_data_fraction=0.5, # full labelled data first
)

baseline_params = TrainParams(
    seed=determinism.seed,
    architecture="resnet50",
    n_epochs=5,
    optimizer=NagParams(
        learning_rate=0.004713842776302303,
        weight_decay=4.287389519996026e-06,
        momentum=0.7222321829989696,
    ),
    freeze_layers=True,
    unfreezing_epochs=(2, 4),
    validation_freq=1,
    time_limit_seconds=None,
    val_acc_target=None,
    unfreeze_last_l_blocks = 2,
    pseudo_threshold = 0.95,
    unsup_weight = 1,
    augmentation = AugmentationParams(
        enabled=False,
        transform=None
    ),
    fixmatch=True,
    use_scheduler=True,
    scheduler_type="plateau"
)

#plot: Compare unsupervised learning for different values of unsup_weight
unsup_weights = [ 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

for unsup_weight in unsup_weights:
    baseline_params.unsup_weight = unsup_weight
    training_params = deepcopy(baseline_params)

    print("\n=====================================")
    print(f"Training with unsup_weight={unsup_weight}")
    print("=====================================")
    # print(training_params.pprint())

    trainer = try_loading_trainer(dataset_params, training_params, determinism)

    test_dataset = load_dataset("test", Trainer.make_base_transform(training_params))
    test_loader = DataLoader(
        test_dataset,
        batch_size=dataset_params.batch_size,
        shuffle=False,
        num_workers=3,
        persistent_workers=False,
        pin_memory=True,
        worker_init_fn=Determinism.data_loader_worker_init_fn(dataset_params.shuffler_seed),
    )

    print(f"Test size: {len(test_loader.dataset)}")
    final_test_acc = evaluate_test_accuracy(trainer, test_loader)
    print(f"Final Test Accuracy: {final_test_acc:.2f}%")



Training with unsup_weight=0.1
Trying to load trainer from disk...
Trainer not found. Retraining...
[Trainer] Unfroze last 2 blocks


Update step:   0%|          | 0/465 [00:00<?, ?it/s]

Epoch [1/5], Loss: 3.2638, Train Acc: 32.47%, Val Acc: 80.65%
Epoch [2/5], Loss: 2.0929, Train Acc: 77.90%, Val Acc: 89.10%
Epoch [3/5], Loss: 0.9740, Train Acc: 86.71%, Val Acc: 91.42%
Epoch [4/5], Loss: 0.5534, Train Acc: 90.58%, Val Acc: 91.14%
Epoch [5/5], Loss: 0.4033, Train Acc: 93.83%, Val Acc: 92.10%
Total elapsed: 99.82s, average per update step: 0.09s
Test size: 3669


Evaluating:   0%|          | 0/115 [00:00<?, ?it/s]

Final Test Accuracy: 89.94%

Training with unsup_weight=0.2
Trying to load trainer from disk...
Trainer not found. Retraining...
[Trainer] Unfroze last 2 blocks


...

Final Test Accuracy: 89.97%

Training with unsup_weight=1.0
Trying to load trainer from disk...
Trainer not found. Retraining...
[Trainer] Unfroze last 2 blocks


Update step:   0%|          | 0/465 [00:00<?, ?it/s]

Epoch [1/5], Loss: 3.2638, Train Acc: 32.47%, Val Acc: 80.65%
Epoch [2/5], Loss: 2.0929, Train Acc: 77.90%, Val Acc: 89.10%
Epoch [3/5], Loss: 0.9740, Train Acc: 86.71%, Val Acc: 91.42%
Epoch [4/5], Loss: 0.5534, Train Acc: 90.58%, Val Acc: 91.14%
Epoch [5/5], Loss: 0.5366, Train Acc: 93.76%, Val Acc: 91.96%
Total elapsed: 98.99s, average per update step: 0.09s
Test size: 3669


Evaluating:   0%|          | 0/115 [00:00<?, ?it/s]

Final Test Accuracy: 89.86%


# Main Testing loop for semi-supervised model

In [17]:
from run import try_loading_trainer
from augmentation import AugmentationParams
from evaluation import evaluate_final_test_accuracy, run_with_different_seeds
from torch.utils.data import DataLoader
from datasets import load_dataset, DatasetParams
from training import Trainer, TrainParams, NagParams
from copy import deepcopy
from caching import invalidate_cache_entry
from determinism import Determinism

%reload_ext autoreload
%autoreload 2


determinism = Determinism(seed=42).sow()  # Keep this before any torch import
   
dataset_params = DatasetParams(
    splitting_seed=determinism.seed,
    shuffler_seed=determinism.seed,
    batch_size=32,
    #trainval_size=None,  # Load all training data
    class_fractions=(1.0,) * 37,
    #trainval_size=400,  # Load a subset
    validation_set_fraction=0.2,  # 20 % of trainval set
    labelled_data_fraction=0.5, # full labelled data first
)

baseline_params = TrainParams(
    seed=determinism.seed,
    architecture="resnet50",
    n_epochs=5,
    optimizer=NagParams(
        learning_rate=0.004713842776302303,
        weight_decay=4.287389519996026e-06,
        momentum=0.7222321829989696,
    ),
    freeze_layers=True,
    unfreezing_epochs=(2, 4),
    validation_freq=1,
    time_limit_seconds=None,
    val_acc_target=None,
    unfreeze_last_l_blocks = 2,
    pseudo_threshold = 0.95,
    unsup_weight = 0.5,
    augmentation = AugmentationParams(
        enabled=False,
        transform=None
    ),
    fixmatch=True,
    use_scheduler=True,
    scheduler_type="plateau"
)

fractions = [0.9] #[0.1, 0.3, 0.5, 0.7, 0.9, 1.0]
epochs_list = [10, 20, 30]

for fraction in fractions:
    dataset_params.labelled_data_fraction = fraction
    for num_epochs in epochs_list:
        training_params = deepcopy(baseline_params)
        training_params.n_epochs = num_epochs

        print("\n=====================================")
        print(f"Training with fraction={fraction}, epochs={num_epochs}")
        print("=====================================")
        # print(training_params.pprint())
        # print(dataset_params.pprint())

        trials = 3
        # run_with_different_seeds(dataset_params, training_params, determinism, trials)
        evaluate_final_test_accuracy(
            dataset_params,
            training_params,
            determinism,
            trials,
            display_misclassified=False,
        )


Training with fraction=0.9, epochs=10
Test size: 3669
Trying to load trainer from disk...


/opt/python/3.10/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Trainer not found. Retraining...
[Trainer] Unfroze last 2 blocks


Update step:   0%|          | 0/930 [00:00<?, ?it/s]

Epoch [1/10], Loss: 2.8015, Train Acc: 50.69%, Val Acc: 86.65%
Epoch [2/10], Loss: 0.9496, Train Acc: 85.60%, Val Acc: 92.10%
Epoch [3/10], Loss: 0.4320, Train Acc: 91.25%, Val Acc: 94.01%
Epoch [4/10], Loss: 0.2839, Train Acc: 93.64%, Val Acc: 94.01%
Epoch [5/10], Loss: 0.2055, Train Acc: 95.61%, Val Acc: 94.41%
Epoch [6/10], Loss: 0.1677, Train Acc: 96.32%, Val Acc: 94.82%
Epoch [7/10], Loss: 0.1382, Train Acc: 97.88%, Val Acc: 95.10%
Epoch [8/10], Loss: 0.1229, Train Acc: 98.10%, Val Acc: 95.10%
Epoch [9/10], Loss: 0.1041, Train Acc: 99.07%, Val Acc: 95.23%
Epoch [10/10], Loss: 0.1104, Train Acc: 99.33%, Val Acc: 95.37%
Total elapsed: 145.67s, average per update step: 0.10s


...

Test Accuracy: 92.314 %
Trying to load trainer from disk...
Trainer not found. Retraining...
[Trainer] Unfroze last 2 blocks


Update step:   0%|          | 0/2790 [00:00<?, ?it/s]

Epoch [1/30], Loss: 2.7900, Train Acc: 51.73%, Val Acc: 86.24%
Epoch [2/30], Loss: 0.9417, Train Acc: 85.37%, Val Acc: 91.83%
Epoch [3/30], Loss: 0.4284, Train Acc: 91.40%, Val Acc: 93.73%
Epoch [4/30], Loss: 0.2909, Train Acc: 93.26%, Val Acc: 94.01%
Epoch [5/30], Loss: 0.2296, Train Acc: 95.53%, Val Acc: 94.41%
Epoch [6/30], Loss: 0.2048, Train Acc: 96.58%, Val Acc: 94.82%
Epoch [7/30], Loss: 0.1612, Train Acc: 97.69%, Val Acc: 94.96%
Epoch [8/30], Loss: 0.1536, Train Acc: 98.33%, Val Acc: 94.82%
Epoch [9/30], Loss: 0.1125, Train Acc: 98.96%, Val Acc: 94.55%
Epoch [10/30], Loss: 0.0768, Train Acc: 99.14%, Val Acc: 95.23%
Epoch [11/30], Loss: 0.1192, Train Acc: 99.14%, Val Acc: 94.41%
Epoch [12/30], Loss: 0.1308, Train Acc: 99.29%, Val Acc: 94.96%
Epoch [13/30], Loss: 0.1139, Train Acc: 99.48%, Val Acc: 94.69%
Epoch [14/30], Loss: 0.1606, Train Acc: 99.48%, Val Acc: 94.96%
Epoch [15/30], Loss: 0.0662, Train Acc: 99.63%, Val Acc: 94.55%
Epoch [16/30], Loss: 0.1059, Train Acc: 99.59%, V

Evaluating:   0%|          | 0/115 [00:00<?, ?it/s]

Test Accuracy: 91.714 %
Test Accuracy Mean: 91.84 %
Test Accuracy Standard Error: 0.24 percentage points


In [ ]:
from run import try_loading_trainer
from evaluation import evaluate_test_accuracy
from torch.utils.data import DataLoader
from datasets import load_dataset
from training import Trainer
from copy import deepcopy

fractions = [0.3, 0.5, 0.7, 0.9, 1.0]
epochs_list = [10, 20, 30, 40]

for fraction in fractions:
    dataset_params.labelled_data_fraction = fraction
    for num_epochs in epochs_list:
        training_params = deepcopy(baseline_params)
        training_params.n_epochs = num_epochs

        print("\n=====================================")
        print(f"Training with fraction={fraction}, epochs={num_epochs}")
        print("=====================================")
        print(training_params.pprint())

        trainer = try_loading_trainer(dataset_params, training_params, determinism)

        test_dataset = load_dataset("test", Trainer.make_base_transform(training_params))
        test_loader = DataLoader(
            test_dataset,
            batch_size=dataset_params.batch_size,
            shuffle=False,
            num_workers=3,
            persistent_workers=False,
            pin_memory=True,
            worker_init_fn=Determinism.data_loader_worker_init_fn(dataset_params.shuffler_seed),
        )

        print(f"Test size: {len(test_loader.dataset)}")
        final_test_acc = evaluate_test_accuracy(trainer, test_loader)
        print(f"Final Test Accuracy: {final_test_acc:.2f}%")

In [ ]:
from evaluation import show_misclassified

final_test_acc, misclassified_samples = evaluate_test_accuracy_and_misclassified(trainer, test_loader, test_dataset)
print(f"Final Test Accuracy: {final_test_acc:.2f}%")
print(f"Number of misclassified samples: {len(misclassified_samples)}")
show_misclassified(misclassified_samples)

In [ ]:
from evaluation import show_misclassified

final_test_acc_tau, misclassified_samples_tau = evaluate_test_accuracy_and_misclassified(trainer, test_loader, test_dataset)
print(f"Final Test Accuracy: {final_test_acc_tau:.2f}%")
print(f"Number of misclassified samples: {len(misclassified_samples_tau)}")
show_misclassified(misclassified_samples_tau)

In [ ]:
misclassified_diff = list(set(misclassified_samples).symmetric_difference(misclassified_samples_tau))
print(f"Difference of misclassified samples: {len(misclassified_diff)}")
show_misclassified(misclassified_diff)

In [ ]:
misclassified_diff2 = list(set(misclassified_samples) - set(misclassified_samples_tau))
print(f"Difference of misclassified samples: {len(misclassified_diff2)}")
show_misclassified(misclassified_diff2)